# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [1]:
import numpy as np
import math
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [19]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

In [55]:
# known values
a = 1.5
t_start = Time1[0]
t_max = Time1[-1]
t_duration = 8
data_time_series = np.array([Time1, Position1])
del_T = np.diff(Time1)[0]
del_T_0 = np.diff(Time1)[0]

# unknown values that are randomly guessed
f = 10
sigma = 10

In [62]:
time_crosscorr, crosscorr_value = san.cross_correlation(del_T_0, t_start, t_max, 
                                                   data_time_series, a, f, sigma, t_duration, del_T)

In [63]:
x = np.argmax(crosscorr_value)
print(Time1[x])

9.56


### Compute a for-loop to run a range of frequencies and sigmas to ouput the maximum value of the cross-correlation.

In [ ]:
# this needs to loop over a range of frequencies and sigmas to output the highest value for the cross-correlation

def freq_sigma_search(t_start, t_max)

result = 0

for i in range(t_start, t_max, 0.01):
    result += 
    
# you can use math.floor and math.ceil to find highest value in an array